# 2048 Game 

 This is an implementation of the popular 2048 puzzle game using Pygame, adapted to run in a Jupyter Notebook.

In [2]:
!pip install pygame

In [3]:
import random
import math
import pygame
from IPython.display import display, clear_output
import ipywidgets as widgets

pygame 2.6.1 (SDL 2.28.4, Python 3.13.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
# Initialize pygame
pygame.init()
pygame.font.init()

In [5]:
# Game constants
FPS = 60
Width, Height = 800, 900
grid_size = 800
rows = 4
cols = 4
rect_height = grid_size // rows
rect_width = grid_size // cols

In [6]:
# Colors
outline_color = (187, 173, 160)
outline_thickness = 10
background_color = (205, 192, 180)
font_color = (119, 110, 101)

In [7]:
# Fonts
FONT = pygame.font.SysFont("comicsans", 60, bold=True)
SCORE_FONT = pygame.font.SysFont("comicsans", 40, bold=True)
MOVE_VEL = 20

In [8]:
# Game state
score = 0
game_over = False
game_won = False

In [9]:
class Tile:
    Colors = [
        (237, 229, 218),
        (238, 225, 201),
        (243, 178, 122),
        (246, 150, 101),
        (247, 124, 95),
        (247, 95, 59),
        (237, 208, 115),
        (237, 204, 99),
        (230, 202, 80)
    ]

    def __init__(self, value, row, col):
        self.value = value
        self.row = row
        self.col = col
        self.x = col * rect_width
        self.y = row * rect_height + Height - grid_size

    def get_color(self):
        color_index = int(math.log2(self.value)) - 1
        color = self.Colors[min(color_index, len(self.Colors) - 1)]
        return color

    def draw(self, window):
        color = self.get_color()
        pygame.draw.rect(window, color, (self.x, self.y, rect_width, rect_height))

        text = FONT.render(str(self.value), 1, font_color)
        window.blit(
            text, 
            (
                self.x + (rect_width/2 - text.get_width()/2),
                self.y + (rect_height/2 - text.get_height()/2), 
            ),
        )

    def set_pos(self, ceil=False):
        if ceil:
            self.row = math.ceil((self.y - (Height - grid_size))/ rect_height)
            self.col = math.ceil(self.x / rect_width)
        else: 
            self.row = math.floor((self.y - (Height - grid_size))/ rect_height)
            self.col = math.floor(self.x / rect_width)    

    def move(self, delta):
        self.x += delta[0]
        self.y += delta[1]

# %%
def draw_grid(window):
    grid_rect = pygame.Rect(0, Height - grid_size, grid_size, grid_size)
    pygame.draw.rect(window, outline_color, grid_rect, outline_thickness)

    for row in range(1, rows):
        y = row * rect_height + Height - grid_size
        pygame.draw.line(window, outline_color, (0, y), (Width, y), outline_thickness)
    
    for col in range(1, cols):
        x = col * rect_width
        pygame.draw.line(window, outline_color, (x, Height - grid_size), (x, Height), outline_thickness)

def draw_score(window):
    score_panel = pygame.Rect(0, 0, Width, Height - grid_size)
    pygame.draw.rect(window, background_color, score_panel)

    score_text = SCORE_FONT.render(f"Score: {score}", 1, font_color)
    window.blit(score_text, (Width // 2 - score_text.get_width() // 2, 
                            (Height - grid_size)//2 - score_text.get_height()//2))

def draw_game_over(window):
    overlay = pygame.Surface((grid_size, grid_size), pygame.SRCALPHA)
    overlay.fill((238, 228, 218, 180))
    window.blit(overlay, (0, Height - grid_size))
    
    text = FONT.render("Game Over!", 1, (119, 110, 101))
    restart_text = SCORE_FONT.render("Press R to restart", 1, (119, 110, 101))

    window.blit(text, (grid_size//2 - text.get_width()//2, Height - grid_size//2 + 50))

def draw_win(window): 
    overlay = pygame.Surface((grid_size, grid_size), pygame.SRCALPHA)
    overlay.fill((237, 194, 46, 180))
    window.blit(overlay,(0, Height - grid_size))

    text = FONT.render("You Win!", 1, (255, 255, 255))
    continue_text = SCORE_FONT.render("Press C to continue", 1, (255, 255, 255))

    window.blit(text, (grid_size//2 - text.get_width()//2, Height - grid_size//2 - text.get_height()//2))

def draw(window, tiles):
    window.fill(background_color)
    draw_score(window)

    for tile in tiles.values():
        tile.draw(window)

    draw_grid(window)

    if game_over:
        draw_game_over(window)
    elif game_won:
        draw_win(window)    
        
    pygame.display.update()

# %%
def get_random_pos(tiles):
    row = None
    col = None
    while True:
        row = random.randrange(0, rows)
        col = random.randrange(0, cols)

        if f"{row}{col}" not in tiles:
            break
    return row, col 

def move_tiles(window, tiles, clock, direction):
    global score, game_won
    updated = True
    block = set()

    if direction == "left":
        sort_func = lambda x: x.col
        reverse = False
        delta = (-MOVE_VEL, 0)
        boundary_check = lambda tile: tile.col == 0
        get_next_tile = lambda tile: tiles.get(f"{tile.row}{tile.col - 1}")
        merge_check = lambda tile, next_tile: tile.x > next_tile.x + MOVE_VEL
        move_check = lambda tile, next_tile: tile.x > next_tile.x + rect_width + MOVE_VEL
        ceil = True

    elif direction == "right":
        sort_func = lambda x: x.col
        reverse = True
        delta = (MOVE_VEL, 0)
        boundary_check = lambda tile: tile.col == cols - 1
        get_next_tile = lambda tile: tiles.get(f"{tile.row}{tile.col + 1}")
        merge_check = lambda tile, next_tile: tile.x < next_tile.x - MOVE_VEL
        move_check = lambda tile, next_tile: tile.x + rect_width + MOVE_VEL < next_tile.x
        ceil = False

    elif direction == "up":   
        sort_func = lambda x: x.row
        reverse = False
        delta = (0, -MOVE_VEL)
        boundary_check = lambda tile: tile.row == 0
        get_next_tile = lambda tile: tiles.get(f"{tile.row - 1}{tile.col}")
        merge_check = lambda tile, next_tile: tile.y > next_tile.y + MOVE_VEL
        move_check = lambda tile, next_tile: tile.y > next_tile.y + rect_height + MOVE_VEL
        ceil = True
        
    elif direction == "down":
        sort_func = lambda x: x.row
        reverse = True
        delta = (0, MOVE_VEL)
        boundary_check = lambda tile: tile.row == rows - 1
        get_next_tile = lambda tile: tiles.get(f"{tile.row + 1}{tile.col}")
        merge_check = lambda tile, next_tile: tile.y < next_tile.y - MOVE_VEL
        move_check = lambda tile, next_tile: tile.y + rect_height + MOVE_VEL < next_tile.y
        ceil = False

    while updated:
        clock.tick(FPS)
        updated = False
        sorted_tiles = sorted(tiles.values(), key=sort_func, reverse=reverse)

        for i, tile in enumerate(sorted_tiles):
            if boundary_check(tile):
                continue

            next_tile = get_next_tile(tile)
            if not next_tile:
                tile.move(delta)
            elif tile.value == next_tile.value and tile not in block and next_tile not in block:
                if merge_check(tile, next_tile):
                    tile.move(delta)    
                else:
                    next_tile.value *= 2
                    score += next_tile.value    
                    block.add(next_tile)
                    if next_tile.value == 2048:
                        game_won = True
                    sorted_tiles.pop(i)
            elif move_check(tile, next_tile):        
                tile.move(delta)
            else: 
                continue   
            
            tile.set_pos(ceil)
            updated = True       
        update_tiles(window, tiles, sorted_tiles)   

    if not game_won and check_game_over(tiles):
        global game_over
        game_over = True
    else:    
        end_move(tiles)   

# %%
def check_game_over(tiles):
    if len(tiles) < rows * cols:
        return False
    
    for tile in tiles.values():
        for dr, dc in [(0, 1), (1, 0), (-1, 0), (0, -1)]:
            new_row, new_col = tile.row + dr, tile.col + dc
            if 0 <= new_row < rows and 0 <= new_col < cols:
                neighbor = tiles.get(f"{new_row}{new_col}")
                if neighbor and neighbor.value == tile.value:
                    return False
    return True            

def end_move(tiles):
    if len(tiles) == 16:
        return
    
    row, col = get_random_pos(tiles)
    tiles[f"{row}{col}"] = Tile(random.choice([2, 4]), row, col)

def update_tiles(window, tiles, sorted_tiles):
    tiles.clear()
    for tile in sorted_tiles:
        tiles[f"{tile.row}{tile.col}"] = tile

    draw(window, tiles)    

def generate_tiles():
    tiles = {}
    for _ in range(2):
        row, col = get_random_pos(tiles)
        tiles[f"{row}{col}"] = Tile(2, row, col)
    return tiles

def reset_game():
    global tiles, score, game_over, game_won
    tiles = generate_tiles()
    score = 0
    game_over = False
    game_won = False

# %%
# Create the game window
WINDOW = pygame.display.set_mode((Width, Height))
pygame.display.set_caption("2048")

# Initialize game state
tiles = generate_tiles()

# Create a button to start the game
start_button = widgets.Button(description="Start Game")
display(start_button)

def start_game(b):
    clear_output()
    print("Game started! Use arrow keys to play. Close the pygame window to stop.")
    
    # Main game loop
    clock = pygame.time.Clock()
    running = True
    
    while running:
        clock.tick(FPS)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
                return
            
            if event.type == pygame.KEYDOWN:
                if not game_over and not game_won:  
                    if event.key == pygame.K_LEFT:
                        move_tiles(WINDOW, tiles, clock, "left")
                    if event.key == pygame.K_RIGHT:
                        move_tiles(WINDOW, tiles, clock, "right")
                    if event.key == pygame.K_UP:
                        move_tiles(WINDOW, tiles, clock, "up")
                    if event.key == pygame.K_DOWN:
                        move_tiles(WINDOW, tiles, clock, "down")
                elif game_over and event.key == pygame.K_r:
                    reset_game()
                elif game_won and event.key == pygame.K_c:
                    game_won = False
        
        draw(WINDOW, tiles)
    
    pygame.quit()

start_button.on_click(start_game)

Button(description='Start Game', style=ButtonStyle())